In [1]:
import skimage
from skimage import io
from skimage.color import rgb2gray
import scipy.ndimage as ndi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import cluster, metrics
from scipy.optimize import minimize_scalar
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform

from scipy.spatial.distance import pdist

import optuna
import time

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 60em; }</style>"))

MAX_ALL_DATA = np.inf

TEST_SIZE = 500


EPS_MIN = 0
EPS_MAX = 18


NR_OBJ = 1
CHECK_NR = 4
GENERAL_TIME = 60
INTERNAL_TIME = 10

In [2]:
def img_center(img):
    center_y, center_x = ndi.center_of_mass(img)
    center_y = round(center_y)
    center_x = round(center_x)
    shape_y, shape_x = img.shape
    top_pad = max((shape_y - 1 - center_y) - center_y, 0)
    bot_pad = max(center_y - (shape_y - 1 - center_y), 0)
    left_pad = max((shape_x - 1 - center_x) - center_x, 0)
    right_pad = max(center_x - (shape_x - 1 - center_x), 0)
    return cv2.copyMakeBorder(img, top_pad, bot_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, None, value=1.)

def img_equalize_size(img, max_shape_x, max_shape_y):
    shape_y, shape_x = img.shape
    top_pad = (max_shape_y - shape_y) // 2
    bot_pad = (max_shape_y - shape_y + 1) // 2
    left_pad = (max_shape_x - shape_x) // 2
    right_pad = (max_shape_x - shape_x + 1) // 2
    return cv2.copyMakeBorder(img, top_pad, bot_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, None, value=1.)

def preprocess_imgs(imgs):
    imgs = imgs.apply(rgb2gray)
    imgs = imgs.apply(img_center)
    max_shape_y = max(imgs.apply(lambda x: x.shape[0]))
    max_shape_x = max(imgs.apply(lambda x: x.shape[1]))
    imgs = imgs.apply(lambda img: img_equalize_size(img, max_shape_x, max_shape_y))
    imgs = imgs.apply(lambda x: np.reshape(x, -1))
    return imgs

initial_df = pd.DataFrame(dtype=object)
initial_df["imgs"] = io.imread_collection('data/*.png', conserve_memory=False)
initial_df["correct"] = pd.read_csv('data/@0CLUSTERING.csv')["Cluster"]
initial_df = initial_df.sample(frac=1).reset_index(drop=True)

if len(initial_df.index) > MAX_ALL_DATA:
    initial_df = initial_df.loc[:MAX_ALL_DATA - 1, :]

initial_df["imgs"] = preprocess_imgs(initial_df["imgs"])

In [3]:
def comp_distance(q):
    res = None
    if q is not None:
        p = np.inf if np.isclose(0, q) else 1/q
        start = time.time()
        res = pd.DataFrame(squareform(pdist(initial_df["imgs"].to_list(), metric="minkowski", p=p)))
        stop = time.time()
        print(f"comp_distance({q}) computed, time: {stop - start}")
    return res






In [4]:
%time comp_distance(0.5)

comp_distance(0.5) computed, time: 67.0220308303833
CPU times: user 1min 5s, sys: 1.39 s, total: 1min 6s
Wall time: 1min 7s


,0,1,2,3,4,5,6,7,8,9,...,7608,7609,7610,7611,7612,7613,7614,7615,7616,7617
0,0.000000,6.761087,8.640704,11.442811,8.736274,6.041618,5.880414,4.993418,5.695164,5.405842,...,8.388748,8.195966,7.460872,7.029647,8.969284,7.646917,7.826741,8.210777,7.626437,5.558866
1,6.761087,0.000000,9.029709,11.166257,8.292422,7.346754,7.270560,6.507365,5.806122,7.068216,...,8.630242,8.619344,9.283629,1.730363,7.598194,8.871518,8.127582,8.641759,9.253296,6.533149
2,8.640704,9.029709,0.000000,11.215333,4.163603,8.651594,8.700928,8.835312,7.559928,9.040720,...,3.379239,5.370515,6.342543,8.715540,9.820928,4.790331,9.589962,5.194174,4.225701,7.249683
3,11.442811,11.166257,11.215333,0.000000,11.072146,11.154013,11.082910,11.351938,11.630512,11.382214,...,11.349006,9.839799,10.373609,11.043928,9.683809,10.247658,10.807343,9.928218,11.084685,11.623656
4,8.736274,8.292422,4.163603,11.072146,0.000000,8.545627,8.560335,8.533878,6.962775,8.883685,...,3.548952,5.947594,6.949350,7.996788,9.595061,5.973926,9.200338,5.816847,5.715268,6.844104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,7.646917,8.871518,4.790331,10.247658,5.973926,8.337030,8.335272,8.151707,7.502562,8.414289,...,5.794365,3.195082,5.640236,8.599808,9.106251,0.000000,8.849570,2.944505,4.429196,7.217638
7614,7.826741,8.127582,9.589962,10.807343,9.200338,8.521945,8.557689,6.982794,8.094123,8.777719,...,9.621820,8.696275,9.342581,8.075992,7.075401,8.849570,0.000000,8.870439,9.309328,8.072046
7615,8.210777,8.641759,5.194174,9.928218,5.816847,8.260577,8.280178,8.299491,7.695878,8.533549,...,6.174050,1.779593,6.281692,8.334226,8.907694,2.944505,8.870439,0.000000,5.551537,7.355429
7616,7.626437,9.253296,4.225701,11.084685,5.715268,7.826135,7.903859,7.777967,7.203009,8.155538,...,4.528308,5.526364,4.505440,8.984487,9.725489,4.429196,9.309328,5.551537,0.000000,6.932859


In [5]:
%time comp_distance(1)

comp_distance(1) computed, time: 126.29124546051025
CPU times: user 1min 16s, sys: 16.4 s, total: 1min 32s
Wall time: 2min 6s


,0,1,2,3,4,5,6,7,8,9,...,7608,7609,7610,7611,7612,7613,7614,7615,7616,7617
0,0.000000,65.976471,88.850980,162.015686,93.905882,56.850980,55.007843,43.058824,51.525490,48.768627,...,88.007843,81.380392,75.415686,70.278431,105.337255,73.596078,82.121569,81.682353,74.137255,48.584314
1,65.976471,0.000000,96.035294,154.674510,86.933333,74.537255,73.509804,63.223529,52.921569,69.215686,...,92.305882,87.380392,105.752941,14.215686,80.647059,92.388235,86.529412,88.137255,101.611765,62.435294
2,88.850980,96.035294,0.000000,149.501961,25.298039,88.329412,89.466667,90.152941,71.145098,94.921569,...,19.541176,32.513725,51.623529,90.729412,114.007843,28.501961,105.592157,30.619608,23.811765,65.450980
3,162.015686,154.674510,149.501961,0.000000,149.082353,152.819608,152.223529,158.211765,165.415686,157.694118,...,154.321569,118.933333,133.713725,152.137255,123.996078,128.498039,146.568627,120.717647,146.898039,165.219608
4,93.905882,86.933333,25.298039,149.082353,0.000000,89.243137,89.807843,87.607843,64.568627,95.317647,...,22.776471,43.552941,62.850980,82.215686,110.419608,45.274510,99.564706,42.074510,42.701961,62.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,73.596078,92.388235,28.501961,128.498039,45.274510,83.050980,83.647059,80.443137,70.352941,83.494118,...,42.262745,12.811765,43.207843,88.643137,100.392157,0.000000,91.898039,11.992157,26.337255,65.239216
7614,82.121569,86.529412,105.592157,146.568627,99.564706,92.305882,93.600000,67.376471,86.690196,96.960784,...,108.850980,87.964706,107.168627,85.764706,72.588235,91.898039,0.000000,91.521569,101.796078,85.733333
7615,81.682353,88.137255,30.619608,120.717647,42.074510,80.705882,81.819608,81.564706,72.588235,84.827451,...,45.533333,4.396078,49.074510,83.552941,95.788235,11.992157,91.521569,0.000000,36.250980,66.603922
7616,74.137255,101.611765,23.811765,146.898039,42.701961,74.752941,76.713725,73.133333,66.149020,79.384314,...,29.909804,35.909804,31.207843,97.349020,112.901961,26.337255,101.796078,36.250980,0.000000,61.474510


In [6]:
%time comp_distance(0)

comp_distance(0) computed, time: 92.16341090202332
CPU times: user 1min 9s, sys: 8.31 s, total: 1min 17s
Wall time: 1min 32s


,0,1,2,3,4,5,6,7,8,9,...,7608,7609,7610,7611,7612,7613,7614,7615,7616,7617
0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.0,1.0,0.454902,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.968627,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.0,1.0,1.000000,1.0,0.0,1.0,1.0,1.0,1.0
7614,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.0,1.0,1.000000,1.0,1.0,0.0,1.0,1.0,1.0
7615,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,0.0,1.0,1.0
7616,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.988235,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0.0,1.0


In [ ]:
%time comp_distance(2.1)

In [ ]:
def get_subsets(df, d):
    ind = df.index.to_list()
    np.random.shuffle(ind)
    ind1 = ind[:TEST_SIZE]
    ind2 = ind[len(df) - TEST_SIZE:]
    np.random.shuffle(ind1)
    np.random.shuffle(ind2)
    d1 = None if d is None else d.loc[ind1, ind1]
    d2 = None if d is None else d.loc[ind2, ind2]
    return [(pd.DataFrame(df, index=ind1, copy=False), d1),
            (pd.DataFrame(df, index=ind2, copy=False), d2)]

def get_subset(df, d):
    sub, _ = get_subsets(df, d)
    return sub

In [ ]:
def clustering(eps, q, data, d):
    #print(f"started clustering({eps}, {q})")
    if np.isclose(q, 0):
        p = np.inf
    else:
        p = 1 / q
    
    if np.isclose(p, 2):
        p = 2
    
    #start = time.time()
    if d is None:
        _, labels = cluster.dbscan(data, eps=eps, p=p, min_samples=1)
    else:
        _, labels = cluster.dbscan(d, eps=eps, min_samples=1, metric='precomputed')

    return labels

In [ ]:
def show_plot(data_x, y):
    pca = PCA(n_components = 2)
    data2D = pca.fit_transform(data_x)
    print(f"nr of classes: {len(set(y))}")
    fig = px.scatter(x=data2D[:, 0], y=data2D[:, 1], color=[str(v) for v in y], width=900, height=600)
    fig.show()


In [ ]:
def objective(eps, q, nr_obj, d, all_data=False):
    if all_data:
        data = initial_df["imgs"].to_list()
        correct = initial_df["correct"].to_list()
        res = clustering(eps, q, data, d)
        return metrics.adjusted_rand_score(correct, res)
    acc = []
    for _ in range(nr_obj):
        for test_df, d1 in get_subsets(initial_df, d):
            data = test_df["imgs"].to_list()
            correct = test_df["correct"].to_list()
            res = clustering(eps, q, data, d1)
            acc.append(metrics.adjusted_rand_score(correct, res))
    return np.mean(acc)

In [ ]:
def check_clustering(eps, q, nr_obj, d):
    test_df, d1 = get_subset(initial_df, d)
    data = test_df["imgs"].to_list()
    correct = test_df["correct"].to_list()
    
    print("correct clustering")
    show_plot(data, correct)
    print("\n")
    
    res = clustering(eps, q, data, d1)
    acc = metrics.adjusted_rand_score(correct, res)
    
    
    print(f"found clustering")
    print(f"eps: {float(eps)}")
    print(f"q: {float(q)}")
    print(f"ACCURACY: {metrics.rand_score(correct, res)}")
    print(f"BALANCE ACCURACY: {acc}")
    
    if nr_obj > 0:
        acc = objective(eps, q, nr_obj, d)

        print(f"AVERAGE BALANCED ACCURACY: {acc}")
    
    show_plot(data, res)
    
    return acc 


In [ ]:
def show_all_data():
    print("all data")
    show_plot(initial_df["imgs"].to_list(), initial_df["correct"].to_list())

show_all_data()

In [ ]:
#def objective_optuna(trial, nr_obj, d):
#    eps = trial.suggest_float("eps", EPS_MIN, EPS_MAX)
#    q = trial.suggest_float("q", 0, 1)
#    return objective(eps, q, nr_obj, d)

In [ ]:
def objective_optuna_eps(trial, q, nr_obj, d, all_data=False):
    eps = trial.suggest_float("eps", EPS_MIN, EPS_MAX)
    return objective(eps, q, nr_obj, d, all_data)

In [ ]:
def get_best_eps(q, nr_obj, internal_timeout, all_data=False):
    d = comp_distance(q)
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_optuna_eps(trial, q, nr_obj, d, all_data), timeout=internal_timeout)
    eps = study.best_trial.params['eps']
    if all_data:
        val = study.best_trial.value
    else:
        val = objective(eps, q, CHECK_NR, d)
    print(f"optimization for {q=} finished")
    print(f"eps: {eps}")
    print(f"value: {val}")
    optuna.visualization.plot_slice(study).show()
    return eps, val

In [ ]:
def objective_optuna_q(trial, nr_obj, internal_timeout, all_data=False):
    q = trial.suggest_float("q", 0, 1)
    eps, val = get_best_eps(q, nr_obj, internal_timeout, all_data)
    return val


In [ ]:
def get_best_q(nr_obj, timeout, internal_timeout, all_data=False):
    study = optuna.create_study(direction="maximize")
    obj = lambda trial: objective_optuna_q(trial, nr_obj, internal_timeout, all_data)
    for
    study.optimize(obj, timeout=timeout)
    print(f"OPTIMIZATION FINISHED!")
    print(f"q: {study.best_trial.params['q']}")
    print(f"value: {study.best_trial.value}")
    optuna.visualization.plot_slice(study).show()
    return study.best_trial.value

In [ ]:
best_q = get_best_q(NR_OBJ, GENERAL_TIME, INTERNAL_TIME, all_data=True)
#print(best_q)

In [ ]:
#check_clustering(2.21, 0.5, 25, None)